In [10]:
import pandas as pd
import geopandas as gpd
from hotaidata.connectdb import ConnectedBigQuery

from shapely.geometry import Point
from rtree import index

In [3]:
# 資料讀取及整理
datalake_db = ConnectedBigQuery("datalake", r"D:/Project/OTHER_token/hotaiconnected-datalake-346603-121e7d34a0bc.json")
leju = datalake_db.read_data("""
    SELECT * 
    FROM `hotaiconnected-datalake-346603.opendata.leju`
""")
leju = gpd.GeoDataFrame(leju, geometry=gpd.GeoSeries.from_wkt(leju['Geometry']))
leju.drop("Geometry", axis=1, inplace=True)
leju['LivingSphereName'] = leju['City'] +'_'+ leju['LivingSphereName']

In [9]:
leju.head(3)

,SId,City,CityCode,Area,PostCode,LivingSphereName,Distance,OneYearAvgPrice,FiveYearAvgPrice,LivingSphereStatus,UpdatedDatetime,geometry
0,10637,台中市,B,清水區,436,台中市_海巡署生活圈,13169.921936,0.00,0.00,2,2022-06-01 18:17:45,"POLYGON ((120.54887 24.28088, 120.54826 24.282..."
1,10640,台中市,B,霧峰區,413,台中市_霧峰市區生活圈,777.081638,17.60,16.21,1,2022-06-01 18:17:45,"POLYGON ((120.71094 24.08128, 120.70950 24.081..."
2,10657,台中市,B,"西屯區,龍井區","407,434",台中市_東海生活圈,5917.291407,29.23,21.43,1,2022-06-01 18:17:45,"POLYGON ((120.59798 24.18931, 120.59344 24.191..."


In [ ]:
def fast_search(x, y, rt_idx, df, col, multi=False):
    """使用 rtree 加速。當經、緯度有一為空值時，直接回傳np.nan。
    反之則會進行比對，並以list或str方式回傳，取決於multi的值。
    Args:
      x: float. 查詢座標的經度
      y: float. 查詢座標的緯度
      rt_idx: rtree.index.Index. 經過rtree設定好的index物件。
      df: dataframe of GeoPandas. 帶有polygon的geo df。
      col: str. geo df中，polygon的對照名稱欄位。
      multi: boolean. polygon是會重疊。這會影響一個查詢座標會不會對應到多的polygon。
        如果設定為True，則會以list回傳，如果設定為False，則會只取第一個比對到的polygon名稱，並以str回傳。
    Return:
      df: dataframe of pandas.
    """
    if np.isnan(x) or np.isnan(y):
        return np.nan
    
    if multi:
        name_list = []
        for poly_id in rt_idx.intersection((x, y)):
            if df.loc[poly_id, 'geometry'].contains(Point(x, y)):
                name_list.append(df.loc[poly_id, col])
            else:
                continue

        if name_list:
            return name_list
        else:
            return np.nan
    
    else:
        for poly_id in rt_idx.intersection((x, y)):
            if df.loc[poly_id, 'geometry'].contains(Point(x, y)):
                return df.loc[poly_id, col]
            
        return np.nan

In [11]:
# 建立 生活圈 polygon 的 rtree index 
leju_idx = index.Index()
for i, row in leju.iterrows():
    leju_idx.insert(i, row['geometry'].bounds)

In [6]:
# orderSession_tmp 中要有經緯度欄位

In [8]:
pick_up_living_sphere = orderSession_tmp.apply(
    lambda x: fast_search(x['PickUpLng'], x['PickUpLat'], leju_idx, leju, 'LivingSphereName'), axis=1
).to_frame(name="PickUp")